# DCGYAN
- Deep Convolutional Generative Adversarial Nets for GYAN

In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Activation
from keras.layers import BatchNormalization, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
import keras.backend as K

import numpy as np
import matplotlib.pyplot as plt

import cv2

In [ ]:
batch_size = 16
z_dim = 100

In [ ]:
pixel_w = 128
pixel_h = 128
channel = 3

x_train = np.load('gyan_images_{}.npy'.format(pixel_w))

for i in range(x_train.shape[0]):
    print(i)
    plt.imshow(x_train[i,:,:,:].astype('uint8'))
    plt.show()

x_train = x_train.astype('float32')
x_train /= 255
x_train = (x_train * 2) -1

## Generator
- Detect start size from output image size.

In [ ]:
def generator_model():
    in_h = int(pixel_h / 4)
    in_w = int(pixel_w / 4)

    model = Sequential()
    model.add(Dense(in_h*in_w*128, input_shape=(z_dim,)))
    model.add(Reshape((in_h, in_w, 128)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
            
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    
    model.add(Conv2D(channel, (3, 3), padding="same"))
    model.add(Activation("tanh"))

    model.summary()
    return model

## Discriminator
- Replace pooling layers with strided-convolutions
- Don't use BatchNormalization because I cannot generate normally.

In [ ]:
def discriminator_model():    
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same', input_shape=(pixel_h, pixel_w, channel)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(64, (3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(128, (3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(256, (3, 3), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(LeakyReLU(0.2))
    
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.summary()
    return model

# Combine Generator and Discriminator

In [ ]:
def generator_and_discriminator(g, d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    model.summary()
    return model

In [ ]:
optimizer = Adam(0.0002, 0.5)

In [ ]:
discriminator = discriminator_model()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])

# Loss Function
- Crossentropy + L1-Norm
- Add regularization term to suppress mode collapse 

$$
L_G = -\frac{1}{N}\sum(t\log{y}) + \frac{\lambda}{N}\sum||a||_1
$$

In [ ]:
delta = 1e-7
r = 0.9
def loss_G(y_true, y_pred):
    return - K.sum(y_true * K.log(y_pred + delta)) / batch_size + r * K.sum(K.abs(y_pred - y_true)) / batch_size

In [ ]:
generator = generator_model()
combined = generator_and_discriminator(generator, discriminator)
combined.compile(loss=loss_G, optimizer=optimizer, metrics=["accuracy"])

In [ ]:
def show_img(epoch, r=5, c=5):
    noise = np.random.normal(0, 1, (r*c, z_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5
    fig, axs = plt.subplots(r, c, figsize=(16, r * 3))
    cnt = 0
    for i in range(r):
        for j in range(c):
            img = np.squeeze(gen_imgs[cnt, :, :, :])
            axs[i, j].imshow(img)
            axs[i, j].axis('off')
            cnt += 1
    plt.show()

In [ ]:
epochs = 10000

half_batch_size = int(batch_size / 2)
d_loss_list = []
d_acc_list = []
g_loss_list = []
g_acc_list = []

for epoch in range(epochs + 1):
    # Discriminator
    idx = np.random.randint(0, x_train.shape[0], half_batch_size)
    imgs = x_train[idx]
    # Create fake images
    noise = np.random.normal(0, 1, (half_batch_size, z_dim))
    gen_imgs = generator.predict(noise)
    # Combine Real + Fake images
    concat_imgs = np.concatenate([imgs, gen_imgs], axis=0)
    # Real label=1、Fake label=0
    concat_labels = np.concatenate([np.ones((half_batch_size, 1)), np.zeros((half_batch_size, 1))], axis=0)
    # train
    d_loss, d_acc = discriminator.train_on_batch(concat_imgs, concat_labels)
    d_loss_list.append(d_loss)
    d_acc_list.append(d_acc)

    # Generator
    noise = np.random.normal(0, 1, (batch_size, z_dim))
    # It wants to look all real.
    g_loss, g_acc = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    g_loss_list.append(g_loss)
    g_acc_list.append(g_acc)

    if epoch % 1000 == 0:
        print("epoch:{} d_loss:{:2f} d_acc:{:2f} g_loss:{:2f} g_acc:{:2f}".format(epoch, d_loss, d_acc, g_loss, g_acc))
        #generator.save_weights("generator_e_{}.hdf5".format(epoch))
        #discriminator.save_weights("discriminator_e_{}.hdf5".format(epoch))
        show_img(epoch, r=2, c=5)

In [ ]:
plt.plot(d_loss_list, label='D Loss')
plt.plot(g_loss_list, label='G Loss')
plt.legend()
plt.show()

In [ ]:
show_img(0, r=5, c=5)

# Generate images while moving latent variables

In [ ]:
noise = np.random.normal(0, 1, (2, z_dim))
start = np.expand_dims(noise[0], axis=0)
end = np.expand_dims(noise[1], axis=0)
steps = 10
start_img = generator.predict(start)
end_img = generator.predict(end)

vectors = []
alpha_values = np.linspace(0, 1, steps)
for alpha in alpha_values:
    vector = start * (1 - alpha) + end * alpha
    vectors.append(vector)
vectors = np.array(vectors)

result_image = []
for i, vec in enumerate(vectors):
    gen_img = np.squeeze(generator.predict(vec), axis=0)
    gen_img = (0.5 * gen_img + 0.5) * 255
    interpolated_img = gen_img.astype('uint8')
    result_image.append(interpolated_img)

result_image = np.array(result_image)
r = 2
c = 5
fig, axs = plt.subplots(r, c, figsize=(16, r * 3))
cnt = 0
for i in range(r):
    for j in range(c):
        img = np.squeeze(result_image[cnt, :, :, :])
        axs[i, j].imshow(img)
        axs[i, j].axis('off')
        cnt += 1
plt.show()   